In [1]:
import pickle
import json
import random
import urllib
import pyproj
import math
import pandas as pd
import numpy as np
import networkx as nx
from scipy import spatial
import requests
from time import sleep
import time
import matplotlib.pyplot as plt

In [3]:
# =============================================================================
# Functions
# =============================================================================

def get_haversine_distance(point_1, point_2):
    """
    Calculate the distance between any 2 points on earth given as [lon, lat]
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(math.radians, [point_1[0], point_1[1], 
                                                point_2[0], point_2[1]])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    r = 6371000 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

def createGridGraphs(grid_coords_ll, graphs, nrows, ncols, cell_size):
    """
    returns new networks including roads around the cells
    cw:
    grid_coords_ll：用于根据grid编号，查询该grid格子左上角的点坐标
    graphs：字典，key是4种交通方式，每个方式对应nodes, edge，以及networkx
    nrows/ncols：grid的行列数，只是便于在将grid点连在一起时计算index
    cell_size：grid尺寸，主要用于计算grid的距离，再计算时间
    结果：把grid加到原来的graph——networkx中，nodes与edges的地理坐标没有加入，只是修改原来的networkx，格网内部以
    九宫格的形式上下左右彼此相连，格网与原来的路网的连接则只发生在格网的4个角上
    """
    for mode in graphs:
#    create graph internal to the grid
    # cw: as a result, it will become a real grid with all neighbours connected bidirected
        graphs[mode]['graph'].add_nodes_from('g'+str(n) for n in range(len(grid_coords_ll))) #cw: add grid nodes to original nx
        for c in range(ncols):
            for r in range(nrows):
                # if not at the end of a row, add h link: horizontal link with the next right node: to right and to left
                if not c==ncols-1:
                    graphs[mode]['graph'].add_edge('g'+str(r*ncols+c), 'g'+str(r*ncols+c+1), 
                          attr_dict={'type': mode, 'weight_minutes':(cell_size/SPEEDS_MET_S[mode])/(60)})
                    graphs[mode]['graph'].add_edge('g'+str(r*ncols+c+1), 'g'+str(r*ncols+c), 
                          attr_dict={'type': mode, 'weight_minutes':(cell_size/SPEEDS_MET_S[mode])/(60)})
                # if not at the end of a column, add v link: vertial links with the next bottom node: to bottom and to top
                if not r==nrows-1:
                    graphs[mode]['graph'].add_edge('g'+str(r*ncols+c), 'g'+str((r+1)*ncols+c), 
                          attr_dict={'type': mode, 'weight_minutes':(cell_size/SPEEDS_MET_S[mode])/(60)})
                    graphs[mode]['graph'].add_edge('g'+str((r+1)*ncols+c), 'g'+str(r*ncols+c), 
                          attr_dict={'type': mode, 'weight_minutes':(cell_size/SPEEDS_MET_S[mode])/(60)})
        # create links between the 4 corners of the grid and the road network
        kd_tree_nodes=spatial.KDTree(np.array(graphs[mode]['nodes'][['x', 'y']]))  #对原网络建立KDTree
        for n in [0, ncols-1, (nrows-1)*ncols, (nrows*ncols)-1]:    #grid4个角点的index
            #根据grid_coords_ll取回每个角点的左上角点坐标，再查询KDTree，返回列表为[dist, nodeIndex]，所以[1]取回nodeIndex
            closest=kd_tree_nodes.query(grid_coords_ll[n], k=1)[1]  
            #上面KDT查询应该也能返回距离，但是是基于平面坐标系计算的，现在要返回球面坐标系距离
            distance_m=get_haversine_distance(grid_coords_ll[n], list(graphs[mode]['nodes'].iloc[closest][['x', 'y']]))
            #加入grid点与最邻近路网节点的edge
            graphs[mode]['graph'].add_edge('g'+str(n), closest, attr_dict={'type': mode, 
                       'weight_minutes':(distance_m/SPEEDS_MET_S[mode])/(60)})
            graphs[mode]['graph'].add_edge(closest, 'g'+str(n), attr_dict={'type': mode, 
                       'weight_minutes':(distance_m/SPEEDS_MET_S[mode])/(60)})
    return graphs 

#def random_points_within(poly, num_points):
#    """ takes a polygon such as an admin boundary or building and selects 
#    a random point inside using rejection sampling
#    """
#    min_x, min_y, max_x, max_y = poly.bounds
#    points = []
#    while len(points) < num_points:
#        random_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
#        if (random_point.within(poly)):
#            points.append(random_point)
#    return points

def approx_shape_centroid(geometry):
    # cw：求形心
    if geometry['type']=='Polygon':
        centroid=list(np.mean(geometry['coordinates'][0], axis=0))
        return centroid
    elif geometry['type']=='MultiPolygon':
        centroid=list(np.mean(geometry['coordinates'][0][0], axis=0))
        return centroid
    else:
        print('Unknown geometry type')
    
def get_simulation_locations(persons):
    """
    For each agent, based on their actual home and work zones (geoids or int grid cells)
    a "simulation" home and a workplace are assigned which may be metagrid cells or portals
    对于每一个agent，我们虽然知道他的实际工作与居住地，但这个系统是geoid系统——要么是普查小区的一长串数字，要么是interactive_grid编号
    我们后面的模拟则是基于meta grid的，所以要把它对应过去，最后的这个home_sim和work_sim就是对应的meta grid编号（内部），
    或者portal（外部）
    """
    for p in persons:  
        for place in ['home', 'work']:
            geoid=p[place+'_geoid']  #该agent的住宅或工作的geoid
            # 如果该geoid是interactive grid，则拿出这个grid编号，通过int_to_meta_grid，查找对应的meta_grid编号
            if 'g' in str(geoid):
                p[place+'_sim']={'type': 'meta_grid', 
                                 'ind': int_to_meta_grid[int(geoid[1:])]}  #这是应该写错了！！
            # 如果该geoid不是interative grid，但是在simulation area zone里（即要模拟的普查小区），则完全随机地从非交互的meta_grid中
            # 抽取一个可能的id，作为他的家或者工作地？？？？这里没有建立sim_area_zone与meta_grid的对应关系啊！！！
            elif p[place+'_geoid'] in sim_area_zone_list:
                relevant_land_use_codes=land_use_codes[place]
                possible_locations=[static_land_uses[rlu] for rlu in relevant_land_use_codes]
                # 上两句：找到工作或居住对应的用地类型——居住就是R，工作就是M或B，然后找到包含R或者包含M/B的所有meta_grid的id
                # 下一句：因为工作有两个M/B，所以上面那个大List实际是上两个小list，下面把它们俩再合并一下，成为一个大List
                possible_locations=[item for sublist in possible_locations for item in sublist]
                p[place+'_sim']={'type': 'meta_grid', 
                                 'ind': random.choice(possible_locations)}
            # 如果该geoid甚至都不在simulation_area_zone中，就不认为是在grid内部的存在，而视为外部的存在，即直接分配到portal上
            # 但是我们有多个Portal啊？？？
            else:
                p[place+'_sim']={'type': 'portal'}


def get_LLs(persons, places):
    """ takes a list of person objects and 
    finds home and work coordinates for them
    modifies in place
    对于geoid形式的实际工作地与实际居住地，获得它们的坐标点：如果是普查区，则是形心+随机误差，如果是interactive_grid，则是左上角点
    """
    for p in persons:  
        for place in places:
            geoid=p[place+'_geoid']
            if 'g' in str(geoid):
                ll=grid_points_ll[int(geoid[1:])]
            else:
                ll=[all_geoid_centroids[geoid][0]+np.random.normal(0, 0.002, 1)[0], 
                    all_geoid_centroids[geoid][1]+np.random.normal(0, 0.002, 1)[0]]
            p[place+'_ll']=ll

def find_route_multi(start_nodes, end_nodes, graph, weight):
    """
    tries to find paths between lists of possible start and end nodes
    Once a path is successfully found it is returned. Otherwise returns None
    输入的start_nodes与end_nodes都是list，其实是一个实际出发地/结束地最近的几个node
    然后只要任何一个start_node与任何一个end_node之间有最短路径，就立即返回，
    只有找遍了所有的可能性都没有路通时，才返回None
    """
    for sn in start_nodes:
        for en in end_nodes:
            try:
                node_path=nx.shortest_path(graph,sn,en, weight=weight)
                return node_path
            except:
                pass
    return None

def get_route_costs(start_nodes, end_nodes, graph, weight):
    node_route=find_route_multi(start_nodes, end_nodes, 
                                            graph, weight)
    """
    输入的start_nodes与end_nodes都是list，其实是一个实际出发地/结束地最近的几个node
    利用上面的find_route_multi找到它们之间的任何一条最短路径，如果一条都没有会传过来一个None
    如果有最短路径——这将是一个路径节点顺序的列表——遍历前后两个节点这间的edge的weight，在这里也就是行程时间mins，即weight_mins，
    同时，每一条edge都有一个type……（奇怪，一开始传进来的graph,即nx，不就是针对特定交通方式的吗，难道说一种交通方式的network里面
    的edge还会有不同的交通方式，比如公交需要步行的配合？），然后，返回的route包括了节点顺序list，包括了每个边的weight的list，
    以及一个汇总weight字典：将上面的weight list按交通方式汇总求合，得到各交通方式的总计mins
    如果没有最短路径，会返回一个类似的没有意义的节点顺序，但是weights会很大，1000min
    
    它返回的信息比较丰富：route本身包括node顺序list，也包含weight
    """
    if node_route:
        weights=[graph[node_route[i]][node_route[i+1]]['attr_dict'][weight] 
            for i in range(len(node_route)-1)]
        types=[graph[node_route[i]][node_route[i+1]]['attr_dict']['type'] 
            for i in range(len(node_route)-1)]
        route={'node_route': node_route,
               'weights': weights}
        for c in ['driving', 'walking', 'waiting',
                  'cycling', 'pt']:
            route[c]=sum([weights[i] for i in range(len(weights)
            ) if types[i]==c])
    else:
        route={'node_route': [end_nodes[0], end_nodes[0]],
               'weights': [0]}
        for c in ['driving', 'walking', 'waiting',
                  'cycling', 'pt']:
            route[c]=1000
    return route
        
def get_routes(persons):
    """ takes a list of person objects 
    and finds the travel time costs of travel by each mode
    modifies in place
    """
    for p in persons:
        p['routes']={}
        start_time=7*60*60+random.choice(range(0,3*60*60))
        if ((p['home_sim']['type']=='meta_grid') and  (p['work_sim']['type']=='meta_grid')):
            p['type']=0 # lives and works on site
            for m in range(4):
                #下面的'ind': index, query的5是返回5个最近值
                """ 
                m：0，1，2，3——用数字带表交通方式
                mode_graphs[m]：将上面的数字转换为英文名称：driving, cycling, walking, pt
                graphs[mode_graphs[m]]：从graphs中用交通方式名称得到对应的交通方式的graph，其中又包括nodes（地理信息）, 
                edges（地理信息），graph（网络拓扑），kdtree（查询树）
                ...['kdtree']：到这里，取出特定交通方式的kdtree，进行query
                p['home_sim']['ind']：个体p的住宅信息'home_sim'是一个字典，'ind'是其中的meta_grid的编号（index）
                meta_grid['features'][p['home_sim']['ind']]：前面的meta_grid['features']是meta_grid是features列表，后面接的编号，
                用于取出个体住宅所在的meta_grid的信息，再后面接['properties']['centroid']，就是按meta_grid的形心位置
                query中逗号后的5，表示在图中查最近的5个节点，后面的[1]是在查询结果中拿来最近节点的编号，而[0]则是距离
                于是home_node_list与word_node_list，就是对于特定的交通方式，取出该方式的网络图，查询特定个体p的工作地与居住地（这里
                都是meta_grid）在图中最近的5个节点的列表
                """
                home_node_list=graphs[mode_graphs[m]]['kdtree'].query(
                        np.array(meta_grid['features'][p['home_sim']['ind']]['properties']['centroid']), 5)[1]
                work_node_list=graphs[mode_graphs[m]]['kdtree'].query(
                        np.array(meta_grid['features'][p['work_sim']['ind']]['properties']['centroid']), 5)[1]
                p['routes'][m]={'route':get_route_costs(home_node_list, work_node_list, 
                                            graphs[mode_graphs[m]]['graph'], 'weight_minutes')}
                p['routes'][m]['sim_start_time']=start_time
        elif p['work_sim']['type']=='meta_grid':
            p['type']=1 # commute_in
            for m in range(4):
                portal_routes={}  #用来存储从各个Portal到达工作地的cost
                best_portal_route_time=float('inf')
                work_node_list=graphs[mode_graphs[m]]['kdtree'].query(
                        np.array(meta_grid['features'][p['work_sim']['ind']]['properties']['centroid']), 5)[1]
                """
                ext_route_costs是从外部读取的关于每种交通方式的路网图节点与外部小区之间成本的信息
                这里ext_route_costs[mode_graphs[m]]是获得特定交通方式的结果
                现在是工作在内，居住在外，我们用已经的居住地geoid（home_geoid）获得从外部居住地到外部portal的成本
                """
                for portal in range(len(ext_route_costs[mode_graphs[m]][str(p['home_geoid'])])):
                    # get route from home zone to portal by this mode
                    # route_to_portal应该是一个字典，是由这个人的实际居住地geoid到这个portal的路径，在driving, waiting,..。上的cost分量
                    route_to_portal=ext_route_costs[mode_graphs[m]][str(p['home_geoid'])][str(portal)] 
                    #portal_routes：字典，由各个Portal到达这个人的工作地的路径的cost
                    portal_routes[portal]=get_route_costs(['p'+str(portal)], work_node_list, 
                                            graphs[mode_graphs[m]]['graph'], 'weight_minutes')
                    for c in ['driving', 'walking', 'waiting',
                              'cycling', 'pt']:
                        portal_routes[portal][c]+=route_to_portal[c]
                    total_time=sum([portal_routes[portal][c] for c in [
                            'driving', 'walking', 'cycling', 'pt']])
                    # 基于driving, waiting,...各分量上的总时间，不断更新这个最优总时间，来确定最佳portal
                    if total_time<best_portal_route_time:
                        best_portal=portal
                        best_portal_route_time=total_time
                p['routes'][m]={'portal': best_portal, 'route': portal_routes[best_portal]}
                # 考虑的好细啊，模拟的出发时间=出发时间+到达portal的时间？可是，由于上面的
                # portal_routes[portal][c]+=route_to_portal[c]，这个时间相当于已经到达work place了吧？？？
                p['routes'][m]['sim_start_time']=int(start_time+best_portal_route_time*60)
        elif p['home_sim']['type']=='meta_grid':
            p['type']=2 # commute_out
            for m in range(4):
                portal_routes={}
                best_portal_route_time=float('inf')
                home_node_list=graphs[mode_graphs[m]]['kdtree'].query(
                        np.array(meta_grid['features'][p['home_sim']['ind']]['properties']['centroid']), 5)[1]
                for portal in range(len(ext_route_costs[mode_graphs[m]][str(p['work_geoid'])])):
                    # get route from home zone to portal by this mode
                    route_from_portal=ext_route_costs[mode_graphs[m]][str(p['work_geoid'])][str(portal)] 
                    portal_routes[portal]=get_route_costs( home_node_list, ['p'+str(portal)],
                                            graphs[mode_graphs[m]]['graph'], 'weight_minutes')
                    for c in ['driving', 'walking', 'waiting',
                              'cycling', 'pt']:
                        portal_routes[portal][c]+=route_from_portal[c]
                    total_time=sum([portal_routes[portal][c] for c in [
                            'driving', 'walking', 'cycling', 'pt']])
                    if total_time<best_portal_route_time:
                        best_portal=portal
                        best_portal_route_time=total_time
                p['routes'][m]={'portal': best_portal, 'route': portal_routes[best_portal]}
                p['routes'][m]['sim_start_time']=start_time
                
def predict_modes(persons):
    """ takes list of person objects and 
    predicts transport modes for each person's commute
    modifies in place
    除了用随机森林预测交通方式之外，对于portal，现在已经使用具体使用了哪种交通方式，就可以调出该交通方式的“best_portal”，
    作为最终选择的portal
    同时，可以调出该交通方式的最佳路径，第一个节点是家，最后一个节点是工作地，并查找出相应的坐标xy，存入个人属性中
    """
    feature_df=pd.DataFrame(persons)  
#    feature_df['bach_degree_yes']=feature_df['SCHL']>20
#    feature_df['bach_degree_no']=~feature_df['bach_degree_yes']
    for feat in ['income', 'age', 'children', 'workers', 'tenure', 'sex', 
                 'bach_degree', 'race', 'cars']:
        new_dummys=pd.get_dummies(feature_df[feat], prefix=feat)
        feature_df=pd.concat([feature_df, new_dummys],  axis=1)
    # TODO: better method of predicting travel times
    # routing engine or feedback from simulation
    feature_df['drive_time_minutes']=  feature_df.apply(lambda row: row['routes'][0]['route']['driving'], axis=1)     
    feature_df['cycle_time_minutes']=  feature_df.apply(lambda row: row['routes'][1]['route']['cycling'], axis=1)     
    feature_df['walk_time_minutes']=  feature_df.apply(lambda row: row['routes'][2]['route']['walking'], axis=1)     
    feature_df['PT_time_minutes']=  feature_df.apply(lambda row: row['routes'][3]['route']['pt'], axis=1)
    feature_df['walk_time_PT_minutes']=feature_df.apply(lambda row: row['routes'][3]['route']['walking'], axis=1)  
    feature_df['drive_time_PT_minutes']=0 
    # TODO: below should come directly from the path-finding
    feature_df['network_dist_km']=feature_df.apply(lambda row: row['drive_time_minutes']*30/60, axis=1) 
    # TODO: change below if modelling housing sales as well
    feature_df['tenure_owned']=False
    feature_df['tenure_other']=False
    feature_df['purpose_HBW']=1
    feature_df['purpose_NHB']=0
    feature_df['purpose_HBO']=0
    feature_df['race_asian']=0
    for rff in rf_features:
        # feature_df[']
        assert rff in feature_df.columns, str(rff) +' not in data.'
#    assert all([rff in feature_df.columns for rff in rf_features]
#    ),"Features in table dont match features in RF model"   
    feature_df=feature_df[rf_features]#reorder columns to match rf model
    mode_probs=mode_rf.predict_proba(feature_df)
    for i,p in enumerate(persons): 
        chosen_mode=int(np.random.choice(range(4), size=1, replace=False, p=mode_probs[i])[0])
        p['mode']=chosen_mode
        if p['home_sim']['type']=='portal': p['home_sim']['ind']=p['routes'][chosen_mode]['portal']
        elif p['work_sim']['type']=='portal': p['work_sim']['ind']=p['routes'][chosen_mode]['portal']
        home_node=p['routes'][chosen_mode]['route']['node_route'][0]
        work_node=p['routes'][chosen_mode]['route']['node_route'][-1]
        p['home_node_ll']=[nodes_xy[chosen_mode][home_node]['x'], 
                           nodes_xy[chosen_mode][home_node]['y']]
        p['work_node_ll']=[nodes_xy[chosen_mode][work_node]['x'], 
                           nodes_xy[chosen_mode][work_node]['y']]
        p['sim_start_time']=p['routes'][chosen_mode]['sim_start_time']

def post_od_data(persons, destination_address):
    od_str=json.dumps([{'home_ll': p['home_node_ll'],
                       'work_ll': p['work_node_ll'],
                       'home_sim': p['home_sim'],
                       'work_sim': p['work_sim'],
                       'type': p['type'],
                       'mode': p['mode'],
                       'start_time': p['sim_start_time']} for p in persons])
    try:
        r = requests.post(destination_address, data = od_str)
        print(r)
        return r
    except requests.exceptions.RequestException as e:
        print('Couldnt send to cityio')
    
#        
def create_trips(persons):
    """ returns a trip objects for each person
    each  trip object contains a list of [lon, lat, timestamp] coordinates
    this is the format required by the deckGL trips layer
    modifies in place
    """
    for p in persons:
        chosen_route=p['routes'][p['mode']]
        route_nodes=chosen_route['node_route']
        route_coords=[nodes_xy[p['mode']][n] for n in route_nodes]
        route_time=[p['sim_start_time']]+[int(p['sim_start_time']+w*60
                    ) for w in np.cumsum(chosen_route['weights'])]
        p['path']=[[int(1e5*route_coords[n]['x'])/1e5, # reduce precision
                    int(1e5*route_coords[n]['y'])/1e5
                    ]for n in range(len(route_nodes))]
        p['timestamps']=route_time
# 
def post_trips_data(persons, destination_address):
    """ posts trip data json to cityIO
    """
    trips_str=json.dumps([{'mode': p['mode'], 'path': p['path'], 
                           'timestamps': p['timestamps']} for p in persons]) 
    try:
        r = requests.post(destination_address, data = trips_str)
        print(r)
    except requests.exceptions.RequestException as e:
        print('Couldnt send to cityio')
#        
#def post_grid_geojson(grid_geo, destination_address):
#    """ posts grid geojson to cityIO
#    """
#    try:
#        r = requests.post(destination_address, data = json.dumps(grid_geo))
#        print(r)
#    except requests.exceptions.RequestException as e:
#        print('Couldnt send grid geojson to cityio')
#        
def create_long_record(person, house, choice_id):
    """ takes a house object and a household object and 
    creates a row for the MNL long data frame 
    """
    beds=min(3, max(1, house['beds']))
    norm_rent=(house['rent']-rent_normalisation['mean'][str(int(beds))])/rent_normalisation['std'][str(int(beds))]
    return {'norm_rent': norm_rent,
            'work_dist': get_haversine_distance(p['work_ll'], h['centroid']),
            'puma_pop_per_sqmeter': house['puma_pop_per_sqmeter'],
            'income_disparity': np.abs(house['puma_med_income']-person['HINCP']),
            'built_since_jan2010': house['built_since_jan2010'],
            'custom_id': person['person_id'],
            'choice_id': choice_id,
            'actual_house_id':house['house_id']}  
def home_location_choices(houses, persons):
    """ takes the house and person objects
    finds the vacant houses and homeless persons
    chooses a housing unit for each person
    modifies the house and person objects in place
    """
    long_data=[]
    # for each household, sample N potential housing choices
    # and add them to the long data frame
    for p in persons:
        #choose N houses
        h_alts=random.sample(houses, 9)
        for hi, h in enumerate(h_alts):
            long_record=create_long_record(p, h, hi+1)
            long_data.append(long_record)             
#             long_data.append(h.long_data_record(hh.hh_income, hh.household_id, hi+1, rent_normalisation))
    long_df=pd.DataFrame(long_data)
    # TODO: why do some houses have nan for norm_rent
    long_df.loc[long_df['norm_rent'].isnull(), 'norm_rent']=0
    long_df['predictions']=home_loc_logit.predict(long_df)
    for p_ind in set(long_df['custom_id']):
        # find maximum prob or sample from probs in subset of long_df
        house_id=np.random.choice(long_df.loc[long_df['custom_id']==p_ind, 'actual_house_id'], 
                                  p=long_df.loc[long_df['custom_id']==p_ind, 'predictions'])
        persons[p_ind]['house_id']=house_id
        persons[p_ind]['home_geoid']=houses[house_id]['home_geoid']
        # update characterictics of persons in these households

## 基本参数

In [24]:
# =============================================================================
# Parameters
# =============================================================================
city='Detroit'
send_to_cityIO=True

# =============================================================================
# Constants
# =============================================================================

ALL_ZONES_PATH='./scripts/cities/'+city+'/clean/model_area.geojson'
SIM_ZONES_PATH='./scripts/cities/'+city+'/clean/sim_zones.json'
# Synthpop results
SIM_POP_PATH='./scripts/cities/'+city+'/clean/sim_pop.json'
VACANT_PATH='./scripts/cities/'+city+'/clean/vacant.json'
FLOATING_PATH='./scripts/cities/'+city+'/clean/floating.json'
# Mode choice model
FITTED_MODE_MODEL_PATH='./scripts/cities/'+city+'/models/trip_mode_rf.p'
RF_FEATURES_LIST_PATH='./scripts/cities/'+city+'/models/rf_features.json'
# Home location choice model
FITTED_HOME_LOC_MODEL_PATH='./scripts/cities/'+city+'/models/home_loc_logit - 1stage.p'
RENT_NORM_PATH='./scripts/cities/'+city+'/models/rent_norm.json'

#Road network graph
PORTALS_PATH='./scripts/cities/'+city+'/clean/portals.geojson'
ROUTE_COSTS_PATH='./scripts/cities/'+city+'/clean/route_costs.json'
SIM_GRAPHS_PATH='./scripts/cities/'+city+'/clean/sim_area_nets.p'

META_GRID_SAMPLE_PATH='./scripts/cities/'+city+'/clean/meta_grid.geojson'
GRID_INT_SAMPLE_PATH='./scripts/cities/'+city+'/clean/grid_interactive.geojson'

# the graph used by each mode
mode_graphs={0:'driving',
             1:'cycling',
             2:'walking',
             3:'pt'}

SPEEDS_MET_S={'driving':30/3.6,
        'cycling':15/3.6,
        'walking':4.8/3.6,
        'pt': 4.8/3.6 # only used for grid use walking speed for pt
        }

kgCO2PerMet={0: 0.45*0.8708/0.00162,
                    1: 0,
                    2: 0,
                    3: 0.45*0.2359/0.00162}#from lbs/mile to US tonnes/m
# TODO: put the below in a text file
# TODO: number of of people per housing cell should vary by type
housing_types={1:{'rent': 800, 'beds': 2, 'built_since_jan2010': True, 
                  'puma_pop_per_sqmeter': 0.000292, 'puma_med_income': 60000,
                  'pop_per_sqmile': 5000, 'tenure': 'rented'},
               2:{'rent': 1500, 'beds': 2, 'built_since_jan2010': True, 
                  'puma_pop_per_sqmeter': 0.000292, 'puma_med_income': 60000,
                  'pop_per_sqmile': 5000, 'tenure': 'rented'}}
# TODO: number of each employment sector for each building type
employment_types= {3:{},4:{}}

land_use_codes={'home': ['R'], 'work': ['M', 'B']}

## 从服务器读取格网数据 

In [23]:
# #cityIO grid data
table_name_map={'Boston':"mocho",
     'Hamburg':"grasbrook",
     'Detroit': "corktown"}
host='https://cityio.media.mit.edu/'
cityIO_grid_url=host+'api/table/'+table_name_map[city]
UPDATE_FREQ=1 # seconds
CITYIO_SAMPLE_PATH='scripts/cities/'+city+'/clean/sample_cityio_data.json' #cityIO backup data

# destination for output files
CITYIO_OUTPUT_PATH=host+'api/table/update/'+table_name_map[city]+'/'

## 读取模型数据和地理信息数据 

In [25]:
# =============================================================================
# Load Data
# =============================================================================
# load the pre-calibrated mode choice model
mode_rf=pickle.load( open( FITTED_MODE_MODEL_PATH, "rb" ) )
rf_features=json.load(open(RF_FEATURES_LIST_PATH, 'r'))
# load the pre-calibrated home location choice model
home_loc_logit=pickle.load( open( FITTED_HOME_LOC_MODEL_PATH, "rb" ) )
rent_normalisation=json.load(open(RENT_NORM_PATH))
#load the network graphs: 4 modes, each with nodes(x,y), edges(roads), and networkx 
graphs=pickle.load(open(SIM_GRAPHS_PATH, 'rb'))
# load the external route costs: ???
ext_route_costs=json.load(open(ROUTE_COSTS_PATH))

# add kdtrees for nodes of each mode
for graph in graphs:
    graphs[graph]['kdtree']=spatial.KDTree(
            np.array(graphs[graph]['nodes'][['x', 'y']]))

# load the zones geojson
all_zones=json.load(open(ALL_ZONES_PATH))
sim_zones=json.load(open(SIM_ZONES_PATH))
portals=json.load(open(PORTALS_PATH))

# add centroids to portals
for p in portals['features']:
    p['properties']['centroid']=approx_shape_centroid(p['geometry'])


if city=='Hamburg':
    geoid_order_all=[f['properties']['GEO_ID'] for f in all_zones['features']]
    sim_area_zone_list=sim_zones
else:
    geoid_order_all=[f['properties']['GEO_ID'].split('US')[1] for f in all_zones['features']]
    sim_area_zone_list=[z.split('US')[1] for z in sim_zones]

all_geoid_centroids={}
# cw: add centroids to zones 
for ind, geo_id in enumerate(geoid_order_all):
#    centroid=shape(all_zones['features'][ind]['geometry']).centroid
    centroid=approx_shape_centroid(all_zones['features'][ind]['geometry'])
#    all_geoid_centroids[geo_id]=[centroid.x, centroid.y]
    all_geoid_centroids[geo_id]=list(centroid)
    
# print(all_geoid_centroids)
# json.dump(all_geoid_centroids, open('C:\0 - MIT\02 - mobility\ford\all_geoid_centroids.p', 'w', encoding='utf-8'))
pickle.dump(all_geoid_centroids, open(r'C:\0 - MIT\02 - mobility\ford\all_geoid_centroids.p', 'wb'))

C:\Users\wangc\Anaconda3\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.20.0 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\wangc\Anaconda3\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.20.0 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\wangc\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:162: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return cls.__new__(cls, **d)


## 处理空间数据

In [12]:
# =============================================================================
# Processing of spatial grid data
# =============================================================================
# Get the grid data
# Interactive grid parameters
# cw: this file is not available !!!
try:
    print(cityIO_grid_url+'/header/spatial')
    with urllib.request.urlopen(cityIO_grid_url+'/header/spatial') as url:
    #get the latest grid data
        cityIO_spatial_data=json.loads(url.read().decode())
except:
    print('Using static cityIO grid file')
    cityIO_data=json.load(open(CITYIO_SAMPLE_PATH))
    cityIO_spatial_data=cityIO_data['header']['spatial']
print(cityIO_spatial_data)

# Interactive grid geojson
# cw: this file does not exists
try:
    print(cityIO_grid_url+'/grid_interactive_area')
    with urllib.request.urlopen(cityIO_grid_url+'/grid_interactive_area') as url:
    #get the latest grid data
        grid_interactive=json.loads(url.read().decode())
except:
    print('Using static cityIO grid file')
    grid_interactive=json.load(open(GRID_INT_SAMPLE_PATH))
print(grid_interactive['features'][20])

try:
    print(cityIO_grid_url+'/meta_grid')
    with urllib.request.urlopen(cityIO_grid_url+'/meta_grid') as url:
    #get the latest grid data
        meta_grid=json.loads(url.read().decode())
except:
    print('Using static cityIO grid file')
    meta_grid=json.load(open(META_GRID_SAMPLE_PATH))
    

https://cityio.media.mit.edu/api/table/corktown/header/spatial
{'cellSize': 30, 'latitude': 42.32783, 'longitude': -83.082279, 'ncols': 16, 'nrows': 13, 'physical_latitude': 42.32783, 'physical_longitude': -83.082279, 'rotation': 23}
https://cityio.media.mit.edu/api/table/corktown/grid_interactive_area
{'geometry': {'coordinates': [[[-83.08079626880229, 42.32800204622321], [-83.08065449192665, 42.32775329082613], [-83.08031925401895, 42.32785849310264], [-83.08046103089458, 42.32810724849973], [-83.08079626880229, 42.32800204622321]]], 'type': 'Polygon'}, 'properties': {'height': 10}, 'type': 'Feature'}
https://cityio.media.mit.edu/api/table/corktown/meta_grid


## 创建交互栅格与大栅格的查询表

In [13]:
# create a lookup from interactive grid to meta_grid
# and a dict of statuc land uses to their locations in the meta_grid
int_to_meta_grid={}
static_land_uses={}
# 有一个小的interactive_grid，还有一个大得多的meta_grid，meta_grid里存有interative_grid的编号，以及当前的用地类型
# 通过遍类meta_grid的每一个grid，建立一个Lookup表，可以由interactive_grid的id查meta_grid的id，同时，对于非交互性的
# grid，它们的用地相当于是不变的，把它们按用地类型归结成一个dict，每个key就是每种用地类型，对应的是一个列表
for fi, f in enumerate(meta_grid['features']):
    if f['properties']['interactive']:
        # cw: interative 
        int_to_meta_grid[int(f['properties']['interactive_id'])]=fi
    else:
        # non-interactive
        this_land_use=f['properties']['land_use']
        if not this_land_use:
            this_land_use='None'
        this_land_use_simple=this_land_use[0]
        if this_land_use_simple in static_land_uses:
            static_land_uses[this_land_use_simple].append(fi)
        else:
            static_land_uses[this_land_use_simple]=[fi]
print(static_land_uses.keys()) #cw
# add centroids to meta_grid_cells
for cell in meta_grid['features']:
    cell['properties']['centroid']=approx_shape_centroid(cell['geometry'])
    
grid_points_ll=[f['geometry']['coordinates'][0][0] for f in grid_interactive['features']]
#cw: a list of leftup points

# x = np.array(grid_interactive['features'][0]['geometry']['coordinates'][0])
# ax = plt.subplot(111)
# ax.plot(x[:,0], x[:,1], 'r-')
# ax.plot(x[0,0], x[0,1], 'b.')
# ax.axis('equal')
# plt.show()

# 将grid连接上原来的graph，即networkx中
graphs=createGridGraphs(grid_points_ll, graphs, cityIO_spatial_data['nrows'], 
                        cityIO_spatial_data['ncols'], cityIO_spatial_data['cellSize'])

#把格网加入原来的sim_area_zone中，注意，以前的sim_area_zone是从all_area_zone的普查小区中切片出一部小区来的
sim_area_zone_list+=['g'+str(i) for i in range(len(grid_points_ll))] 



dict_keys(['N', 'R', 'B', 'S', 'M', 'T', 'P'])


## Networkx最短路径Debug 

In [17]:
%matplotlib qt
# visulization of network
mode = 'driving'
# mode = 'pt'
nodes = graphs[mode]['nodes']
edges = graphs[mode]['edges']
network = graphs[mode]['graph']
exchangeEdges = []
for e_n1, e_n2 in network.edges:
    if str(e_n1).startswith('g') and (not str(e_n2).startswith('g') and not str(e_n2).startswith('p')):
        exchangeEdges.append((e_n1, e_n2))
    elif (not str(e_n1).startswith('g') and not str(e_n1).startswith('p')) and str(e_n2).startswith('g'):
        exchangeEdges.append((e_n1, e_n2))
# print(exchangeEdges)
for e_n1, e_n2 in exchangeEdges:
    print('from {} to {}: {}'.format(e_n1, e_n2, network[e_n1][e_n2]['attr_dict']))
nodesIDList_1 = [x for x in list(network.nodes) if not str(x).startswith('g')]
nodesIDList_2 = [x for x in list(network.nodes) if str(x).startswith('g')]
sn, en = np.random.choice(nodesIDList_1), np.random.choice(nodesIDList_2)
sn, en = nodesIDList_2[0], nodesIDList_2[-1]
sn, en = 'g0', 'g13'
# sn, en = '103', '240'
if not (sn.startswith('g') or sn.startswith('p')) :
    sn = int(sn)
if not (en.startswith('g') or en.startswith('p')):
    en = int(en)
print('\nstart node = {}, end node = {}'.format(sn, en))
node_path=nx.shortest_path(network,sn,en, weight='weight_minutes')
print(node_path)
tt_mins = 0
for id in range(len(node_path)-1):
    print('from {} to {}: {}'.format(node_path[id], node_path[id+1], network[node_path[id]][node_path[id+1]]['attr_dict']))
    tt_mins += network[node_path[id]][node_path[id+1]]['attr_dict']['weight_minutes']
print('Total minutes: {}'.format(tt_mins))
ax = plt.subplot(111)
ax.axis('off')
ax.axis('equal')
x, y = np.array(nodes['x']), np.array(nodes['y'])
ax.scatter(x, y)
for id, row in edges.iterrows():
    try:
        fromx, fromy = nodes.loc[nodes['id']==row['from'], 'x'].values[0], nodes.loc[nodes['id']==row['from'], 'y'].values[0]
        tox, toy = nodes.loc[nodes['id']==row['to'], 'x'].values[0], nodes.loc[nodes['id']==row['to'], 'y'].values[0]
        ax.plot([fromx, tox], [fromy, toy], 'b-')
    except:
        print('Do not find node: from = {}, to = {}'.format(row['from'], row['to']))
        continue

for nodeID in range(len(node_path)-1):
    try:
        fromx, fromy = nodes.loc[nodes['node_id']==node_path[nodeID], 'x'].values[0], nodes.loc[nodes['node_id']==node_path[nodeID], 'y'].values[0]
        tox, toy = nodes.loc[nodes['node_id']==node_path[nodeID+1], 'x'].values[0], nodes.loc[nodes['node_id']==node_path[nodeID+1], 'y'].values[0]
        ax.plot([fromx, tox], [fromy, toy], 'r-')
    except:
        print('Can not find route {} - {}'.format(node_path[nodeID], node_path[nodeID+1]))

plt.show()

manualRoute = ['g'+str(n*16) for n in range(13)]
manualRoute = ['g'+str(n) for n in range(16)]
print('\nManual Route: {}'.format(manualRoute))
tt_minutes_g = 0
for id in range(len(manualRoute)-1):
    print('from {} to {}: {}'.format(manualRoute[id], manualRoute[id+1], network[manualRoute[id]][manualRoute[id+1]]['attr_dict']))
    tt_minutes_g += network[manualRoute[id]][manualRoute[id+1]]['attr_dict']['weight_minutes']
print('Total minutes: {}'.format(tt_minutes_g))

from 103 to g192: {'type': 'driving', 'weight_minutes': 0.04151551239965241}
from 114 to g15: {'type': 'driving', 'weight_minutes': 0.1051525505997706}
from 173 to g207: {'type': 'driving', 'weight_minutes': 0.07643933580135939}
from 240 to g0: {'type': 'driving', 'weight_minutes': 0.0030435240765307884}

start node = g0, end node = g13
['g0', 240, 63, 64, 172, 171, 29, 28, 170, 114, 'g15', 'g14', 'g13']
from g0 to 240: {'type': 'driving', 'weight_minutes': 0.0030435240765307884}
from 240 to 63: {'weight_minutes': 0.20237310999999997, 'type': 'driving'}
from 63 to 64: {'weight_minutes': 0.276987192, 'type': 'driving'}
from 64 to 172: {'weight_minutes': 0.030436137999999998, 'type': 'driving'}
from 172 to 171: {'weight_minutes': 0.21098030799999998, 'type': 'driving'}
from 171 to 29: {'weight_minutes': 0.097213758, 'type': 'driving'}
from 29 to 28: {'weight_minutes': 0.20607110399999998, 'type': 'driving'}
from 28 to 170: {'weight_minutes': 0.09545469, 'type': 'driving'}
from 170 to 114

In [18]:
# =============================================================================
# Node Locations
# =============================================================================
#对于每种交通方式，把原来的graph中的nodes与外部出入口portal放在一起，编成一个新的nodes集，各自的序号为在原先的dataframe中的自然序号
#另外，用了0,1,2,3代表4种交通方式的英文名，好像没啥必要
# create a list of nodes with their coords for each mode
nodes_xy={}
for mode in mode_graphs:
    nodes_xy[mode]={}
    for i in range(len(graphs[mode_graphs[mode]]['nodes'])):
        nodes_xy[mode][i]={'x':graphs[mode_graphs[mode]]['nodes'].iloc[i]['x'],
             'y':graphs[mode_graphs[mode]]['nodes'].iloc[i]['y']}
#    for i in range(len(grid_points_ll)):
#        nodes_xy[mode]['g'+str(i)]={'x':grid_points_ll[i][0],
#             'y':grid_points_ll[i][1]}
    for p in range(len(portals['features'])):
#        p_centroid=shape(portals['features'][p]['geometry']).centroid
        p_centroid=approx_shape_centroid(portals['features'][p]['geometry'])
#        nodes_xy[mode]['p'+str(p)]={'x':p_centroid.x,
#             'y':p_centroid.y}
        nodes_xy[mode]['p'+str(p)]={'x':p_centroid[0],
             'y':p_centroid[1]}

## 处理基础模拟人口 

In [20]:
# =============================================================================
# Population
# =============================================================================

# load sim_persons
base_sim_persons=json.load(open(SIM_POP_PATH))
# load floaters
base_floating_persons=json.load(open(FLOATING_PATH))
# load vacant houses
base_vacant_houses=json.load(open(VACANT_PATH))
for h in base_vacant_houses:
    h['centroid']=all_geoid_centroids[h['home_geoid']]

if base_sim_persons: 
    get_simulation_locations(base_sim_persons)
    get_LLs(base_sim_persons, ['home', 'work'])
    get_routes(base_sim_persons)
    predict_modes(base_sim_persons)
    post_od_data(base_sim_persons, CITYIO_OUTPUT_PATH+'od')
#    create_trips(base_sim_persons)
#    post_trips_data(base_sim_persons, CITYIO_OUTPUT_PATH+'trips')

<Response [200]>


## 实时模拟

In [32]:
lastId=0
while True:
# 检查数据是否更新
    try:
#         print('hash url: {}'.format(cityIO_grid_url+'/meta/hashes/grid'))
        with urllib.request.urlopen(cityIO_grid_url+'/meta/hashes/grid') as url:
            hash_id=json.loads(url.read().decode())
    except:
        print('Cant access cityIO')
        hash_id=1
    print('hash = ', hash_id)
    if hash_id==lastId:
        print('No new scenario')
        sleep(1)
    else:
        try:
#             print('grid data url: {}'.format(cityIO_grid_url+'/grid'))
            with urllib.request.urlopen(cityIO_grid_url+'/grid') as url:
                cityIO_grid_data=json.loads(url.read().decode())
        except:
            print('Using static cityIO grid file')
            cityIO_data=json.load(open(CITYIO_SAMPLE_PATH)) # cw: not available 
            cityIO_grid_data=cityIO_data['grid']
        start_time=time.time()
        lastId=hash_id
## =============================================================================
##         FAKE DATA FOR SCENAIO EXPLORATION
##        cityIO_grid_data=[[int(i)] for i in np.random.randint(3,5,len(cityIO_grid_data))] # all employment
##        cityIO_grid_data=[[int(i)] for i in np.random.randint(1,3,len(cityIO_grid_data))] # all housing
        cityIO_grid_data=[[int(i)] for i in np.random.randint(1,5,len(cityIO_grid_data))] # random mix
        print(cityIO_grid_data)
##        cityIO_grid_data=[[int(i)] for i in np.random.randint(2,4,len(cityIO_grid_data))] # affordable + employment
## =============================================================================
        new_houses=[]
        new_persons=[]
        
        # 新增住宅
        for ht in housing_types:
            ht_locs=[i for i in range(len(cityIO_grid_data)) if cityIO_grid_data[i][0]==ht]
            for htl in ht_locs:
                add_house=housing_types[ht].copy()
                add_house['home_geoid']='g'+str(htl)
                add_house['centroid']=grid_points_ll[htl]
                new_houses.append(add_house)          
        
        # 新增就业用地 = Homeless
        random.seed(0)
        for et in employment_types:
            et_locs=[i for i in range(len(cityIO_grid_data)) if cityIO_grid_data[i][0]==et]
            for etl in et_locs:
                add_person=random.choice(base_floating_persons).copy()
                add_person['work_geoid']='g'+str(etl)
                new_persons.append(add_person)
                random.seed(0)
        
        # 所有需要居住地选择的人
        floating_persons=base_floating_persons+new_persons
        get_LLs(floating_persons, ['work'])
        # 所有的空置住宅
        vacant_houses=base_vacant_houses+new_houses
        for ip, p in enumerate(floating_persons):
            p['person_id']=ip
        for ih, h in enumerate(vacant_houses):
            h['house_id']=ih
        hlc_data = {'houses': vacant_houses, 'persons': floating_persons}
        
        # 居住地选择
        home_location_choices(vacant_houses, floating_persons)
        new_sim_persons=[p for p in floating_persons if
                         (p['home_geoid'] in sim_area_zone_list or
                          p['work_geoid'] in sim_area_zone_list)]
        # 获得居住地与就业地的大栅格
        get_simulation_locations(new_sim_persons)
        get_LLs(new_sim_persons, ['home', 'work'])
        get_routes(new_sim_persons)   # 获得home-work路径
        predict_modes(new_sim_persons)   # 预测交通模式
        post_od_data(base_sim_persons+ new_sim_persons, CITYIO_OUTPUT_PATH+'od')  # 上传OD数据至服务器
#        create_trips(new_sim_persons)
#        post_trips_data(base_sim_persons+ new_sim_persons, CITYIO_OUTPUT_PATH+'trips')
        finish_time=time.time()
        print('Response time: '+ str(finish_time-start_time))
        sleep(0.2)

hash =  73fa231cdf3e90453c478cb9fa018fb69e525419c598a38493db030be95d9435
[[3], [2], [1], [3], [1], [4], [4], [1], [2], [3], [3], [4], [1], [3], [4], [1], [3], [1], [3], [3], [4], [4], [3], [3], [2], [3], [3], [2], [3], [4], [1], [3], [3], [4], [3], [3], [3], [1], [3], [4], [1], [1], [2], [2], [3], [3], [2], [2], [4], [4], [3], [1], [3], [2], [3], [4], [1], [1], [2], [1], [2], [3], [1], [2], [4], [2], [3], [4], [3], [1], [4], [1], [1], [1], [1], [4], [2], [1], [4], [3], [3], [2], [3], [1], [2], [2], [4], [2], [4], [4], [3], [1], [2], [4], [1], [2], [1], [4], [2], [1], [1], [3], [3], [3], [3], [2], [3], [3], [3], [3], [2], [1], [4], [3], [3], [2], [4], [1], [2], [2], [2], [4], [3], [2], [3], [4], [3], [4], [4], [2], [1], [1], [2], [1], [2], [3], [3], [2], [1], [2], [4], [3], [1], [2], [4], [3], [3], [2], [1], [2], [3], [1], [4], [2], [1], [4], [2], [2], [1], [1], [2], [2], [3], [3], [3], [4], [4], [1], [4], [1], [1], [3], [1], [4], [4], [1], [3], [1], [1], [1], [3], [4], [4], [3], [3], [

C:\Users\wangc\Anaconda3\lib\site-packages\pylogit\choice_tools.py:703: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  design_matrix = np.hstack((x[:, None] for x in independent_vars))


Response time: 0.7254314422607422
hash =  73fa231cdf3e90453c478cb9fa018fb69e525419c598a38493db030be95d9435
No new scenario
hash =  73fa231cdf3e90453c478cb9fa018fb69e525419c598a38493db030be95d9435
No new scenario
hash =  73fa231cdf3e90453c478cb9fa018fb69e525419c598a38493db030be95d9435
No new scenario
hash =  73fa231cdf3e90453c478cb9fa018fb69e525419c598a38493db030be95d9435
No new scenario
hash =  73fa231cdf3e90453c478cb9fa018fb69e525419c598a38493db030be95d9435
No new scenario
hash =  73fa231cdf3e90453c478cb9fa018fb69e525419c598a38493db030be95d9435
No new scenario
hash =  73fa231cdf3e90453c478cb9fa018fb69e525419c598a38493db030be95d9435
No new scenario
hash =  73fa231cdf3e90453c478cb9fa018fb69e525419c598a38493db030be95d9435
No new scenario
hash =  73fa231cdf3e90453c478cb9fa018fb69e525419c598a38493db030be95d9435
No new scenario
hash =  73fa231cdf3e90453c478cb9fa018fb69e525419c598a38493db030be95d9435
No new scenario
hash =  73fa231cdf3e90453c478cb9fa018fb69e525419c598a38493db030be95d9435
No 

KeyboardInterrupt: 